In [ ]:
from openai import OpenAI
import requests
import json
from functions import convert_pdfs_to_pngs, get_the_files_with_paths, encode_image, validate_json
import os
import re
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
# Załaduj zmienne środowiskowe z pliku .env
load_dotenv()

# Przypisz zmienne środowiskowe do zmiennych
api_key = os.getenv('OPENAI_API')
pdf_path = os.getenv('PDF_DIRECTORY')
png_path = os.getenv('OUTPUT_DIRECTORY')
main_json_file = os.getenv('JSON_MAIN_FILE')

In [ ]:
# Konwertowanie PDF'ów na PNG z folderu PDFs do folderu PNGs jak i zmienianie im nazwy na prefix 'Converted' i datę utworzenia pliku.
convert_pdfs_to_pngs(pdf_path, png_path)

# Wybieranie plików do OCR'owania z katalogu PNG, które nie zaczynają się od DONE
files_to_analyze = get_the_files_with_paths(png_path)

print(f"Liczba plików do Sprawdzenia: {len(files_to_analyze)}")

In [ ]:
# Jeden pliczek do testów na razie
file  = files_to_analyze[-2]
file

In [ ]:
# Getting the base64 string for GPT
base64_image = encode_image(file)

OCR and Image Recognition with GPT

In [ ]:
prompt = """
Jesteś analizazatorem paragonów. Na podstawie podanego zdjęcia paragonu, wyciągnij z niego jak najwiecej informacji.
Korzystaj z polskich znaków, tam gdzie jest to wymagane.
Odpowiadaj tylko i wyłącznie w postaci JSONA, korzystając z ponizszego szablonu:
{
  "transactions": [
    {
      "id": "unique_transaction_id_1",  // Unikalny identyfikator transakcji
      "receipt_number": "1234567890",  // Numer paragonu
      "timestamp": "2023-09-23T14:55:00Z",  // Znacznik czasu transakcji w formacie ISO 8601
      "date": "2023-09-23",  // Data transakcji
      "store": {
        "name": "Dealz",  // Nazwa sklepu
        "address": "Ul. Dąbrowskiego 79a, Poznań",  // Adres sklepu
        "additional_information": "Sklep: P2339",  // Dodatkowe informacje o sklepie
        "NIP": "7811963267"  // Numer identyfikacji podatkowej sklepu. Jeśli pojawią się tutaj znaki '-', pomiń je i wpisz same cyfry.
      },
      "cashier": "John Doe",  // Imię i nazwisko kasjera
      "terminal_id": "POS12345",  // Identyfikator terminala płatniczego
      "total": 53.00,  // Całkowita kwota transakcji
      "taxes_total": 8.23, // Suma wszystkich podatków w transakcji
      "currency": "PLN",  // Waluta transakcji
      "payment_method": "credit_card",  // Metoda płatności
      "taxes": [
        {
          "type": "VAT",  // Rodzaj podatku
          "rate": 0.23,  // Stawka podatku
          "amount": 8.23  // Kwota podatku
        }
      ],
      "discounts": [
        {
          "description": "Weekend discount",  // Opis zniżki
          "amount": 5.00  // Kwota zniżki
        }
      ],
      "items": [
        {
		  "name": "3008948 wilg. chus.samochod.biodegradowalne C&c 1*4.49",  // Nazwa bezpośrednio 1:1 z paragoniu, bez żadnych przeinaczeń
	      "name_simplified": "wilgotne chusteczki do samochododu", // Nazwa uproszczona,
          "unit_price": {
            "before_discount": 4.49,  // Cena jednostkowa przed zniżką
            "after_discount": 4.49  // Cena jednostkowa po zniżce
          },
          "quantity": 1,  // Ilość jednostek produktu
          "total_price": {
            "before_discount": 4.49,  // Całkowita cena przed zniżką
            "after_discount": 4.49  // Całkowita cena po zniżce
          },
          "unit_of_measurement": "szt.",  // Jednostka miary produktu
          "quantity_in_units": 1,  // Ilość produktu w gramach (1000 gram to 1 kg)
          "category": "Samochód",  // Kategoria produktu
          "tax_rate": 0.23,  // Stawka podatku
          "discount": {
            "description": null,  // Opis zniżki na produkt
            "amount": 0  // Kwota zniżki na produkt
          },
          "manufacturer": null,  // Producent produktu
          "comment": null  // Komentarz do produktu
        },
        {
        	"name": "Filet z p.kurczaka",  // Nazwa bezpośrednio 1:1 z paragoniu, bez żadnych przeinaczeń
	      "name_simplified": "Filet z piersi kurczaka", // Nazwa uproszczona,
          "unit_price": {
            "before_discount": 22.00,  // Cena jednostkowa przed zniżką
            "after_discount": 22.00  // Cena jednostkowa po zniżce
          },
          "quantity": 1,  // Ilość jednostek produktu
          "total_price": {
            "before_discount": 10.71,  // Całkowita cena przed zniżką
            "after_discount": 10.71  // Całkowita cena po zniżce
          },
          "unit_of_measurement": "g",  // Jednostka miary produktu
          "quantity_in_units": 0.487,  // Ilość produktu w gramach (1000 gram to 1 kg)
          "category": "Mięso",  // Kategoria produktu
          "tax_rate": 0.05,  // Stawka podatku
          "discount": {
            "description": null,  // Opis zniżki na produkt
            "amount": 0  // Kwota zniżki na produkt
          },
          "manufacturer": null,  // Producent produktu
          "comment": null  // Komentarz do produktu
        }
    ],
      "comments": "Zakupy",  // Komentarz do transakcji
      "category": "Zakupy Spożywcze",  // Kategoria całej transakcji
      "transaction_comment": "Weekendowe Zakupy"  // Komentarz do całej transakcji
    }
  ]
}
```
"""

In [ ]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ]
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
# Get the interesting part
content = response.json()['choices'][0]['message']['content']

# Remove the ```json and ``` markers
cleaned_content = content.strip('```json\n').strip('\n```')

# Parse the cleaned string as JSON
json_data = json.loads(cleaned_content)

# Print the parsed JSON object to verify
print(json.dumps(json_data, indent=4, ensure_ascii=False))

In [ ]:
validate_json(json_data)